In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from IPython.display import display, Image, clear_output
import time
import random
random.seed(42)

#send_to_google_form(data_dict, form_url)
def send_to_google_form(data_dict, form_url):
    
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [ ]:
print("Welcome to the Memory Test!") 
time.sleep(2)
clear_output(wait=False)

data_consent_info = """DATA CONSENT INFORMATION:
Please read:
we wish to record your response data
to an anonymised public data repository.
Your data will be used for educational teaching purposes
practising data analysis and visualisation.
Please type yes in the box below if you consent to the upload."""

print(data_consent_info)

result = input("> ")

if result == "yes":
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")

else:
    raise(Exception("User did not consent to continue test."))

clear_output(wait=False)


id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:

- two letters based on the initials (first and last name) of a childhood friend

- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise

then your unique identifer would be CBTC

"""

print(id_instructions)

user_id = input("> ")

print("User entered id:", user_id)

clear_output(wait=False)


print('How old are you?')
age = input("> ")
clear_output(wait=False)

print('Which gender are you?')
gender = input("> ")
clear_output(wait=False)


def display_message(message, delay=2):
    print(message)
    time.sleep(delay)
    clear_output(wait=False)

def display_level_and_image(level, image_path, time_to_memorize):
    display_message(f"Level {level}")
    display_message(f"You have {time_to_memorize} seconds to memorize this image.")
    display(Image(image_path, width=300))
    time.sleep(time_to_memorize)
    clear_output(wait=False)

def ask_question(question, choices, correct_choice, answers_list):
    print(question)
    for choice in choices:
        print(choice)
    
    user_answer = input().lower()
    
    if user_answer == correct_choice:
        print("Correct! Well done!")
    else:
        print(f"Wrong! The correct answer is: {correct_choice}")

    answers_list.append(user_answer)  # Append the user's answer to the list
    
    time.sleep(2)
    clear_output(wait=False)
    
    return user_answer == correct_choice

def ask_questions(questions_and_answers, num_questions, answers_list):
    correct_answers = 0
    for question, choices, correct_choice in random.sample(questions_and_answers, num_questions):
        if ask_question(question, choices, correct_choice, answers_list):
            correct_answers += 1
    
    return correct_answers

def main():
    correct_answers = 0
    answers_list = []

    random.seed(42)
    
    levels = [
        ("Lv.1.jpg", 5, [("What is the shape below the circle?", ["a) square", "b) triangle", "c) circle"], "a")]),
        ("Lv.2.jpg", 5, [
            ("What is the shape above the triangle?", ["a) square", "b) cross", "c) triangle"], "b"),
            ("What is the color of the circle?", ["a) red", "b) blue", "c) green"], "a")
        ]),
        ("Lv.3.jpg", 7, [
            ("What is the shape in the middle column?", ["a) star", "b) circle", "c) square"], "a"),
            ("What is the shape below the star?", ["a) circle", "b) exclamation mark", "c) question mark"], "c")
        ]),
        ("Lv.4.jpg", 7, [
            ("What is the shape in the middle column?", ["a) star", "b) triangle", "c) square"], "b"),
            ("What is the shape above the star?", ["a) question mark", "b) exclamation mark", "c) star"], "b"),
            ("What is the color of the moon?", ["a) green", "b) blue", "c) red"], "a")
        ]),
        ("Lv.5.jpg", 10, [
            ("What is the shape in the middle column?", ["a) star", "b) equal sign", "c) division sign"], "c"),
            ("What is the shape above the star?", ["a) question mark", "b) exclamation mark", "c) star"], "b"),
            ("What is the color of the percentage sign?", ["a) green", "b) blue", "c) red"], "c")
        ]),
        ("Lv.6.jpg", 10, [
            ("What is the shape in the upper left column?", ["a) star", "b) exclamation mark", "c) division sign"], "b"),
            ("What is the shape above the star?", ["a) question mark", "b) exclamation mark", "c) cross"], "c"),
            ("What is the color of the AT sign (@)?", ["a) green", "b) blue", "c) red"], "a"),
            ("What is the color of the equal sign?", ["a) green", "b) blue", "c) red"], "c")
        ]),
        ("Lv.7.jpg", 15, [
            ("What is the shape in the upper right column?", ["a) star", "b) exclamation mark", "c) division sign"], "b"),
            ("What is the shape below the plus sign?", ["a) question mark", "b) exclamation mark", "c) cross"], "a"),
            ("What is the color of the AT sign (@)?", ["a) green", "b) blue", "c) red"], "c"),
            ("What is the color of the equal sign?", ["a) green", "b) blue", "c) red"], "a"),
        ])   
    ]

    for level, (image_path, time_to_memorize, questions_and_answers) in enumerate(levels, start=1):
        display_level_and_image(level, image_path, time_to_memorize)
        correct_answers += ask_questions(questions_and_answers, len(questions_and_answers), answers_list)
        
    print(f"You've got {correct_answers}/{sum(len(questions) for _, _, questions in levels)} answers correct!")

    total_questions = sum(len(questions) for _, _, questions in levels)
    accuracy = (correct_answers / total_questions) * 100
    print(f"Your accuracy is: {accuracy:.2f}%")

    # Answer Dictionary
    ans_dict = {
        'Participant': user_id,
        'Age': age,
        'Gender': gender,
        **{f'Answer {i+1}': answers_list[i] for i in range(len(answers_list))},
        'Accuracy': accuracy
    }

    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSe68LhUp899P5BZwOqQtGiVm4dlHlsrh-bpPWx8eyiV-AF-cw/viewform'
    send_to_google_form(ans_dict, form_url)

if __name__ == "__main__":
    main()

You've got 16/19 answers correct!
Your accuracy is: 84.21%
